The basic task is to predict the price of a house depending on the features given. Results will be evaluated using RMSE.

In [22]:
#importing libraries
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', None) # making sure to print all the columns present in the df

# Data analysis and pre-processing

In [2]:
df= pd.read_csv('./data/train.csv')

In [3]:
print(df.shape)
df.head()

(1460, 81)


Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
1         Lvl    AllPub       FR2       Gtl      Veenker      Feedr   
2         Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
3         Lvl    AllPub    Corner       Gtl      Crawfor       Norm   
4         Lvl    AllPub       FR2       Gtl      NoRidge       Norm   

  Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0       Norm     1Fam     2Story            7            5       2003   
1       Norm     1Fam     1Story            6            8       1976   
2       Norm     1Fam     2Story            7            5       2001   
3       Norm     1Fam     2Story            7            5       1915   
4       Norm     1Fam     2Story            8            5       2000   

   YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0          2003     Gable  CompShg     VinylSd     VinylSd    BrkFace   
1          1976     Gable  CompShg     MetalSd     MetalSd       None   
2          2002     Gable  CompShg     VinylSd     VinylSd    BrkFace   
3          1970     Gable  CompShg     Wd Sdng     Wd Shng       None   
4          2000     Gable  CompShg     VinylSd     VinylSd    BrkFace   

   MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond BsmtExposure  \
0       196.0        Gd        TA      PConc       Gd       TA           No   
1         0.0        TA        TA     CBlock       Gd       TA           Gd   
2       162.0        Gd        TA      PConc       Gd       TA           Mn   
3         0.0        TA        TA     BrkTil       TA       Gd           No   
4       350.0        Gd        TA      PConc       Gd       TA           Av   

  BsmtFinType1  BsmtFinSF1 BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0          GLQ         706          Unf           0        150          856   
1          ALQ         978          Unf           0        284         1262   
2          GLQ         486          Unf           0        434          920   
3          ALQ         216          Unf           0        540          756   
4          GLQ         655          Unf           0        490         1145   

  Heating HeatingQC CentralAir Electrical  1stFlrSF  2ndFlrSF  LowQualFinSF  \
0    GasA        Ex          Y      SBrkr       856       854             0   
1    GasA        Ex          Y      SBrkr      1262         0             0   
2    GasA        Ex          Y      SBrkr       920       866             0   
3    GasA        Gd          Y      SBrkr       961       756             0   
4    GasA        Ex          Y      SBrkr      1145      1053             0   

   GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  BedroomAbvGr  \
0       1710             1             0         2         1             3   
1       1262             0             1         2         0             3   
2       1786             1             0         2         1             3   
3       1717             1             0         1         0             3   
4       2198             1             0         2         1             4   

   KitchenAbvGr KitchenQual  TotRmsAbvGrd Functional  Fireplaces FireplaceQu  \
0             1          Gd             8        Typ           0         NaN   
1             1          TA             6        Typ           1          TA   
2             1          Gd             6        Typ           1          TA   
3             1          Gd            

In [4]:
# from a quick look, it can be seen that there are many NaN values

# counting number of NaNs in all the columns

# as there are 81 columns, to see all the values use to_string() and sort_values() and to get a clearer picture

na_count = df.isna().sum().sort_values(ascending=False)

# printing columns with atleast 1 missing value
na_count = na_count[na_count>0]
print(na_count.to_string())

PoolQC          1453
MiscFeature     1406
Alley           1369
Fence           1179
FireplaceQu      690
LotFrontage      259
GarageYrBlt       81
GarageCond        81
GarageType        81
GarageFinish      81
GarageQual        81
BsmtFinType2      38
BsmtExposure      38
BsmtQual          37
BsmtCond          37
BsmtFinType1      37
MasVnrArea         8
MasVnrType         8
Electrical         1


In [5]:
# analyzing the columns and reading data description to see how to treat the missing values 
# print(df.LotFrontage.value_counts().to_string())
df.loc[df.LotFrontage.isna(),'LotFrontage'] = 0
df.loc[df.MasVnrArea.isna(),'MasVnrArea'] = 0

df.loc[df.FireplaceQu.isna(),'FireplaceQu'] = 'No_FP'
df.loc[df.Alley.isna(),'Alley'] = 'No_alley'
df.loc[df.PoolQC.isna(),'PoolQC'] = 'No_pool'
df.loc[df.MiscFeature.isna(),'MiscFeature'] = 'No_MF'
df.loc[df.Fence.isna(),'Fence'] = 'No_Fence'
df.loc[df.BsmtExposure.isna(),'BsmtExposure'] = 'No_Bment'
df.loc[df.BsmtQual.isna(),'BsmtQual'] = 'No_Bment'
df.loc[df.BsmtFinType2.isna(),'BsmtFinType2'] = 'No_Bment'
df.loc[df.BsmtFinType1.isna(),'BsmtFinType1'] = 'No_Bment'
df.loc[df.BsmtCond.isna(),'BsmtCond'] = 'No_Bment'

In [6]:
na_count_after = df.isna().sum().sort_values(ascending=False)
na_count_after = na_count_after[na_count_after>0]
print(na_count_after.to_string())

GarageFinish    81
GarageYrBlt     81
GarageType      81
GarageQual      81
GarageCond      81
MasVnrType       8
Electrical       1


In [7]:
# checking if garage attributes are missing for same obersvations

_ = df[df.GarageFinish.isna()].loc[:,['GarageFinish','GarageYrBlt','GarageQual','GarageCond','GarageType']]
print(_.to_string())

     GarageFinish  GarageYrBlt GarageQual GarageCond GarageType
39            NaN          NaN        NaN        NaN        NaN
48            NaN          NaN        NaN        NaN        NaN
78            NaN          NaN        NaN        NaN        NaN
88            NaN          NaN        NaN        NaN        NaN
89            NaN          NaN        NaN        NaN        NaN
99            NaN          NaN        NaN        NaN        NaN
108           NaN          NaN        NaN        NaN        NaN
125           NaN          NaN        NaN        NaN        NaN
127           NaN          NaN        NaN        NaN        NaN
140           NaN          NaN        NaN        NaN        NaN
148           NaN          NaN        NaN        NaN        NaN
155           NaN          NaN        NaN        NaN        NaN
163           NaN          NaN        NaN        NaN        NaN
165           NaN          NaN        NaN        NaN        NaN
198           NaN          NaN        Na

In [8]:
# as the garage features are missing for the observations where garage is not present, we can impute them all with same value
df.loc[df.GarageFinish.isna(),'GarageFinish'] = 'No_Garage'
df.loc[df.GarageYrBlt.isna(),'GarageYrBlt'] = 0
df.loc[df.GarageQual.isna(),'GarageQual'] = 'No_Garage'
df.loc[df.GarageCond.isna(),'GarageCond'] = 'No_Garage'
df.loc[df.GarageType.isna(),'GarageType'] = 'No_Garage'

na_count_after = df.isna().sum().sort_values(ascending=False)
na_count_after = na_count_after[na_count_after>0]
print(na_count_after.to_string())

MasVnrType    8
Electrical    1


In [9]:
# for MasVnrType and Electrical, there is no clear path to what can be done. 
# Therefore, I will use KNNImputer 
# I could have filled them with most frequent element too. However, I think KNNImputer works the best

'''  I WILL TAKE CARE OF TTHOSE TWO MISSING VALUES LATER'''

'  I WILL TAKE CARE OF TTHOSE TWO MISSING VALUES LATER'

In [10]:
# looking at the data to find outliers
# we can also use techniques like finding the interquartile range / z-score
# we can also try to visualize the data using the box plot and find the outliers, however, as there are 37 numeric columns, making box plots might not be the ideal solution
df.describe()

Id   MSSubClass  LotFrontage        LotArea  OverallQual  \
count  1460.000000  1460.000000  1460.000000    1460.000000  1460.000000   
mean    730.500000    56.897260    57.623288   10516.828082     6.099315   
std     421.610009    42.300571    34.664304    9981.264932     1.382997   
min       1.000000    20.000000     0.000000    1300.000000     1.000000   
25%     365.750000    20.000000    42.000000    7553.500000     5.000000   
50%     730.500000    50.000000    63.000000    9478.500000     6.000000   
75%    1095.250000    70.000000    79.000000   11601.500000     7.000000   
max    1460.000000   190.000000   313.000000  215245.000000    10.000000   

       OverallCond    YearBuilt  YearRemodAdd   MasVnrArea   BsmtFinSF1  \
count  1460.000000  1460.000000   1460.000000  1460.000000  1460.000000   
mean      5.575342  1971.267808   1984.865753   103.117123   443.639726   
std       1.112799    30.202904     20.645407   180.731373   456.098091   
min       1.000000  1872.000000   1950.000000     0.000000     0.000000   
25%       5.000000  1954.000000   1967.000000     0.000000     0.000000   
50%       5.000000  1973.000000   1994.000000     0.000000   383.500000   
75%       6.000000  2000.000000   2004.000000   164.250000   712.250000   
max       9.000000  2010.000000   2010.000000  1600.000000  5644.000000   

        BsmtFinSF2    BsmtUnfSF  TotalBsmtSF     1stFlrSF     2ndFlrSF  \
count  1460.000000  1460.000000  1460.000000  1460.000000  1460.000000   
mean     46.549315   567.240411  1057.429452  1162.626712   346.992466   
std     161.319273   441.866955   438.705324   386.587738   436.528436   
min       0.000000     0.000000     0.000000   334.000000     0.000000   
25%       0.000000   223.000000   795.750000   882.000000     0.000000   
50%       0.000000   477.500000   991.500000  1087.000000     0.000000   
75%       0.000000   808.000000  1298.250000  1391.250000   728.000000   
max    1474.000000  2336.000000  6110.000000  4692.000000  2065.000000   

       LowQualFinSF    GrLivArea  BsmtFullBath  BsmtHalfBath     FullBath  \
count   1460.000000  1460.000000   1460.000000   1460.000000  1460.000000   
mean       5.844521  1515.463699      0.425342      0.057534     1.565068   
std       48.623081   525.480383      0.518911      0.238753     0.550916   
min        0.000000   334.000000      0.000000      0.000000     0.000000   
25%        0.000000  1129.500000      0.000000      0.000000     1.000000   
50%        0.000000  1464.000000      0.000000      0.000000     2.000000   
75%        0.000000  1776.750000      1.000000      0.000000     2.000000   
max      572.000000  5642.000000      3.000000      2.000000     3.000000   

          HalfBath  BedroomAbvGr  KitchenAbvGr  TotRmsAbvGrd   Fireplaces  \
count  1460.000000   1460.000000   1460.000000   1460.000000  1460.000000   
mean      0.382877      2.866438      1.046575      6.517808     0.613014   
std       0.502885      0.815778      0.220338      1.625393     0.644666   
min       0.000000      0.000000      0.000000      2.000000     0.000000   
25%       0.000000      2.000000      1.000000      5.000000     0.000000   
50%       0.000000      3.000000      1.000000      6.000000     1.000000   
75%       1.000000      3.000000      1.000000      7.000000     1.000000   
max       2.000000      8.000000      3.000000     14.000000     3.000000   

       GarageYrBlt   GarageCars   GarageArea   WoodDeckSF  OpenPorchSF  \
count  1460.000000  1460.000000  1460.000000  1460.000000  1460.000000   
mean   1868.739726     1.767123   472.980137    94.244521    46.660274   
std     453.697295     0.747315   213.804841   125.338794    66.256028   
min       0.000000     0.000000     0.000000     0.000000     0.000000   
25%    1958.000000     1.000000   334.500000     0.000000     0.000000   
50%    1977.000000     2.000000   480.000000     0.000000    25.000000   
75%    2001.000000     2.000000   576.000000   168.000000    68.000000   
max    2

In [11]:
df.drop('Id',inplace = True,axis = 1)
df

MSSubClass MSZoning  LotFrontage  LotArea Street     Alley LotShape  \
0             60       RL         65.0     8450   Pave  No_alley      Reg   
1             20       RL         80.0     9600   Pave  No_alley      Reg   
2             60       RL         68.0    11250   Pave  No_alley      IR1   
3             70       RL         60.0     9550   Pave  No_alley      IR1   
4             60       RL         84.0    14260   Pave  No_alley      IR1   
...          ...      ...          ...      ...    ...       ...      ...   
1455          60       RL         62.0     7917   Pave  No_alley      Reg   
1456          20       RL         85.0    13175   Pave  No_alley      Reg   
1457          70       RL         66.0     9042   Pave  No_alley      Reg   
1458          20       RL         68.0     9717   Pave  No_alley      Reg   
1459          20       RL         75.0     9937   Pave  No_alley      Reg   

     LandContour Utilities LotConfig LandSlope Neighborhood Condition1  \
0            Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
1            Lvl    AllPub       FR2       Gtl      Veenker      Feedr   
2            Lvl    AllPub    Inside       Gtl      CollgCr       Norm   
3            Lvl    AllPub    Corner       Gtl      Crawfor       Norm   
4            Lvl    AllPub       FR2       Gtl      NoRidge       Norm   
...          ...       ...       ...       ...          ...        ...   
1455         Lvl    AllPub    Inside       Gtl      Gilbert       Norm   
1456         Lvl    AllPub    Inside       Gtl       NWAmes       Norm   
1457         Lvl    AllPub    Inside       Gtl      Crawfor       Norm   
1458         Lvl    AllPub    Inside       Gtl        NAmes       Norm   
1459         Lvl    AllPub    Inside       Gtl      Edwards       Norm   

     Condition2 BldgType HouseStyle  OverallQual  OverallCond  YearBuilt  \
0          Norm     1Fam     2Story            7            5       2003   
1          Norm     1Fam     1Story            6            8       1976   
2          Norm     1Fam     2Story            7            5       2001   
3          Norm     1Fam     2Story            7            5       1915   
4          Norm     1Fam     2Story            8            5       2000   
...         ...      ...        ...          ...          ...        ...   
1455       Norm     1Fam     2Story            6            5       1999   
1456       Norm     1Fam     1Story            6            6       1978   
1457       Norm     1Fam     2Story            7            9       1941   
1458       Norm     1Fam     1Story            5            6       1950   
1459       Norm     1Fam     1Story            5            6       1965   

      YearRemodAdd RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType  \
0             2003     Gable  CompShg     VinylSd     VinylSd    BrkFace   
1             1976     Gable  CompShg     MetalSd     MetalSd       None   
2             2002     Gable  CompShg     VinylSd     VinylSd    BrkFace   
3             1970     Gable  CompShg     Wd Sdng     Wd Shng       None   
4             2000     Gable  CompShg     VinylSd     VinylSd    BrkFace   
...            ...       ...      ...         ...         ...        ...   
1455          2000     Gable  CompShg     VinylSd     VinylSd       None   
1456          1988     Gable  CompShg     Plywood     Plywood      Stone   
1457          2006     Gable  CompShg     CemntBd     CmentBd       None   
1458          1996       Hip  CompShg     MetalSd     MetalSd       None   
1459          1965     Gable  CompShg     HdBoard     HdBoard       None   

      MasVnrArea ExterQual ExterCond Foundation BsmtQual BsmtCond  \
0          196.0        Gd        TA      PConc       Gd       TA   
1            0.0        TA        TA     CBlock       Gd       TA   
2          162.0        Gd        TA      PConc       Gd       TA   
3            0.0        TA        TA     BrkTil       TA       Gd   
4          350.0     

In [12]:
# as the columns are not string. We will have to change the object datatype into string datatype
# we can use .astype('|S') to convert the object to string depending on the max length of strings encountered

cat_feat = []
for col in df:
    if ((df[col].dtype)) == 'object':
        cat_feat.append(col)
#         df[col] = df[col].astype('string') 
    else:
        df[col] = df[col].astype('float64')
        
df.GarageYrBlt = df.GarageYrBlt.astype('float64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   float64
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1460 non-null   float64
 3   LotArea        1460 non-null   float64
 4   Street         1460 non-null   object 
 5   Alley          1460 non-null   object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   float64
 17  OverallCond    1460 non-null   float64
 18  YearBuil

In [13]:
df_cat = df[cat_feat]
df_num = df.drop(cat_feat,axis = 1)

In [14]:
# df_cat.info()
df_cat

MSZoning Street     Alley LotShape LandContour Utilities LotConfig  \
0          RL   Pave  No_alley      Reg         Lvl    AllPub    Inside   
1          RL   Pave  No_alley      Reg         Lvl    AllPub       FR2   
2          RL   Pave  No_alley      IR1         Lvl    AllPub    Inside   
3          RL   Pave  No_alley      IR1         Lvl    AllPub    Corner   
4          RL   Pave  No_alley      IR1         Lvl    AllPub       FR2   
...       ...    ...       ...      ...         ...       ...       ...   
1455       RL   Pave  No_alley      Reg         Lvl    AllPub    Inside   
1456       RL   Pave  No_alley      Reg         Lvl    AllPub    Inside   
1457       RL   Pave  No_alley      Reg         Lvl    AllPub    Inside   
1458       RL   Pave  No_alley      Reg         Lvl    AllPub    Inside   
1459       RL   Pave  No_alley      Reg         Lvl    AllPub    Inside   

     LandSlope Neighborhood Condition1 Condition2 BldgType HouseStyle  \
0          Gtl      CollgCr       Norm       Norm     1Fam     2Story   
1          Gtl      Veenker      Feedr       Norm     1Fam     1Story   
2          Gtl      CollgCr       Norm       Norm     1Fam     2Story   
3          Gtl      Crawfor       Norm       Norm     1Fam     2Story   
4          Gtl      NoRidge       Norm       Norm     1Fam     2Story   
...        ...          ...        ...        ...      ...        ...   
1455       Gtl      Gilbert       Norm       Norm     1Fam     2Story   
1456       Gtl       NWAmes       Norm       Norm     1Fam     1Story   
1457       Gtl      Crawfor       Norm       Norm     1Fam     2Story   
1458       Gtl        NAmes       Norm       Norm     1Fam     1Story   
1459       Gtl      Edwards       Norm       Norm     1Fam     1Story   

     RoofStyle RoofMatl Exterior1st Exterior2nd MasVnrType ExterQual  \
0        Gable  CompShg     VinylSd     VinylSd    BrkFace        Gd   
1        Gable  CompShg     MetalSd     MetalSd       None        TA   
2        Gable  CompShg     VinylSd     VinylSd    BrkFace        Gd   
3        Gable  CompShg     Wd Sdng     Wd Shng       None        TA   
4        Gable  CompShg     VinylSd     VinylSd    BrkFace        Gd   
...        ...      ...         ...         ...        ...       ...   
1455     Gable  CompShg     VinylSd     VinylSd       None        TA   
1456     Gable  CompShg     Plywood     Plywood      Stone        TA   
1457     Gable  CompShg     CemntBd     CmentBd       None        Ex   
1458       Hip  CompShg     MetalSd     MetalSd       None        TA   
1459     Gable  CompShg     HdBoard     HdBoard       None        Gd   

     ExterCond Foundation BsmtQual BsmtCond BsmtExposure BsmtFinType1  \
0           TA      PConc       Gd       TA           No          GLQ   
1           TA     CBlock       Gd       TA           Gd          ALQ   
2           TA      PConc       Gd       TA           Mn          GLQ   
3           TA     BrkTil       TA       Gd           No          ALQ   
4           TA      PConc       Gd       TA           Av          GLQ   
...        ...        ...      ...      ...          ...          ...   
1455        TA      PConc       Gd       TA           No          Unf   
1456        TA     CBlock       Gd       TA           No          ALQ   
1457        Gd      Stone       TA       Gd           No          GLQ   
1458        TA     CBlock       TA       TA           Mn          GLQ   
1459        TA     CBlock       TA       TA           No          BLQ   

     BsmtFinType2 Heating HeatingQC CentralAir Electrical KitchenQual  \
0             Unf    GasA        Ex          Y      SBrkr          Gd   
1             Unf    GasA        Ex          Y      SBrkr          TA   
2             Unf    GasA        Ex          Y      SBrkr          Gd   
3             Unf    GasA        Gd          Y      SBrkr          Gd   
4             Unf    GasA        Ex          Y      SBrkr          Gd   
...           ...     ...       ...        ... 

In [15]:
# one hot encoding the categorical features 
# using sklearn implementation instead of pandas dummies as categories might vary depending on the train/test data
enc = OneHotEncoder(handle_unknown='ignore' )
enc.fit(df_cat) # we will use this same fitted encoder to fit the test dataset
print(enc.categories_)
_ =  enc.transform(df_cat).toarray()
new_cols = enc.get_feature_names()
df_cat_encoded = pd.DataFrame(_,columns = new_cols)

[array(['C (all)', 'FV', 'RH', 'RL', 'RM'], dtype=object), array(['Grvl', 'Pave'], dtype=object), array(['Grvl', 'No_alley', 'Pave'], dtype=object), array(['IR1', 'IR2', 'IR3', 'Reg'], dtype=object), array(['Bnk', 'HLS', 'Low', 'Lvl'], dtype=object), array(['AllPub', 'NoSeWa'], dtype=object), array(['Corner', 'CulDSac', 'FR2', 'FR3', 'Inside'], dtype=object), array(['Gtl', 'Mod', 'Sev'], dtype=object), array(['Blmngtn', 'Blueste', 'BrDale', 'BrkSide', 'ClearCr', 'CollgCr',
       'Crawfor', 'Edwards', 'Gilbert', 'IDOTRR', 'MeadowV', 'Mitchel',
       'NAmes', 'NPkVill', 'NWAmes', 'NoRidge', 'NridgHt', 'OldTown',
       'SWISU', 'Sawyer', 'SawyerW', 'Somerst', 'StoneBr', 'Timber',
       'Veenker'], dtype=object), array(['Artery', 'Feedr', 'Norm', 'PosA', 'PosN', 'RRAe', 'RRAn', 'RRNe',
       'RRNn'], dtype=object), array(['Artery', 'Feedr', 'Norm', 'PosA', 'PosN', 'RRAe', 'RRAn', 'RRNn'],
      dtype=object), array(['1Fam', '2fmCon', 'Duplex', 'Twnhs', 'TwnhsE'], dtype=object), array(

In [16]:
df_cat_encoded

x0_C (all)  x0_FV  x0_RH  x0_RL  x0_RM  x1_Grvl  x1_Pave  x2_Grvl  \
0            0.0    0.0    0.0    1.0    0.0      0.0      1.0      0.0   
1            0.0    0.0    0.0    1.0    0.0      0.0      1.0      0.0   
2            0.0    0.0    0.0    1.0    0.0      0.0      1.0      0.0   
3            0.0    0.0    0.0    1.0    0.0      0.0      1.0      0.0   
4            0.0    0.0    0.0    1.0    0.0      0.0      1.0      0.0   
...          ...    ...    ...    ...    ...      ...      ...      ...   
1455         0.0    0.0    0.0    1.0    0.0      0.0      1.0      0.0   
1456         0.0    0.0    0.0    1.0    0.0      0.0      1.0      0.0   
1457         0.0    0.0    0.0    1.0    0.0      0.0      1.0      0.0   
1458         0.0    0.0    0.0    1.0    0.0      0.0      1.0      0.0   
1459         0.0    0.0    0.0    1.0    0.0      0.0      1.0      0.0   

      x2_No_alley  x2_Pave  x3_IR1  x3_IR2  x3_IR3  x3_Reg  x4_Bnk  x4_HLS  \
0             1.0      0.0     0.0     0.0     0.0     1.0     0.0     0.0   
1             1.0      0.0     0.0     0.0     0.0     1.0     0.0     0.0   
2             1.0      0.0     1.0     0.0     0.0     0.0     0.0     0.0   
3             1.0      0.0     1.0     0.0     0.0     0.0     0.0     0.0   
4             1.0      0.0     1.0     0.0     0.0     0.0     0.0     0.0   
...           ...      ...     ...     ...     ...     ...     ...     ...   
1455          1.0      0.0     0.0     0.0     0.0     1.0     0.0     0.0   
1456          1.0      0.0     0.0     0.0     0.0     1.0     0.0     0.0   
1457          1.0      0.0     0.0     0.0     0.0     1.0     0.0     0.0   
1458          1.0      0.0     0.0     0.0     0.0     1.0     0.0     0.0   
1459          1.0      0.0     0.0     0.0     0.0     1.0     0.0     0.0   

      x4_Low  x4_Lvl  x5_AllPub  x5_NoSeWa  x6_Corner  x6_CulDSac  x6_FR2  \
0        0.0     1.0        1.0        0.0        0.0         0.0     0.0   
1        0.0     1.0        1.0        0.0        0.0         0.0     1.0   
2        0.0     1.0        1.0        0.0        0.0         0.0     0.0   
3        0.0     1.0        1.0        0.0        1.0         0.0     0.0   
4        0.0     1.0        1.0        0.0        0.0         0.0     1.0   
...      ...     ...        ...        ...        ...         ...     ...   
1455     0.0     1.0        1.0        0.0        0.0         0.0     0.0   
1456     0.0     1.0        1.0        0.0        0.0         0.0     0.0   
1457     0.0     1.0        1.0        0.0        0.0         0.0     0.0   
1458     0.0     1.0        1.0        0.0        0.0         0.0     0.0   
1459     0.0     1.0        1.0        0.0        0.0         0.0     0.0   

      x6_FR3  x6_Inside  x7_Gtl  x7_Mod  x7_Sev  x8_Blmngtn  x8_Blueste  \
0        0.0        1.0     1.0     0.0     0.0         0.0         0.0   
1        0.0        0.0     1.0     0.0     0.0         0.0         0.0   
2        0.0        1.0     1.0     0.0     0.0         0.0         0.0   
3        0.0        0.0     1.0     0.0     0.0         0.0         0.0   
4        0.0        0.0     1.0     0.0     0.0         0.0         0.0   
...      ...        ...     ...     ...     ...         ...         ...   
1455     0.0        1.0     1.0     0.0     0.0         0.0         0.0   
1456     0.0        1.0     1.0     0.0     0.0         0.0         0.0   
1457     0.0        1.0     1.0     0.0     0.0         0.0         0.0   
1458     0.0        1.0     1.0     0.0     0.0         0.0         0.0   
1459     0.0        1.0     1.0     0.0     0.0         0.0         0.0   

      x8_BrDale  x8_BrkSide  x8_ClearCr  x8_CollgCr  x8_Crawfor  x8_Edwards  \
0           0.0         0.0         0.0         1.0         0.0         0.0   
1           0.0         0.0         0.0         0.0         0.0         0.0   
2           0.0         0.0         0.0         1.0         0.0         0.0   
3           0.0         0.

In [17]:
encoded_df = pd.concat([df_cat_encoded,df_num],axis = 1)
encoded_df.head(3)

x0_C (all)  x0_FV  x0_RH  x0_RL  x0_RM  x1_Grvl  x1_Pave  x2_Grvl  \
0         0.0    0.0    0.0    1.0    0.0      0.0      1.0      0.0   
1         0.0    0.0    0.0    1.0    0.0      0.0      1.0      0.0   
2         0.0    0.0    0.0    1.0    0.0      0.0      1.0      0.0   

   x2_No_alley  x2_Pave  x3_IR1  x3_IR2  x3_IR3  x3_Reg  x4_Bnk  x4_HLS  \
0          1.0      0.0     0.0     0.0     0.0     1.0     0.0     0.0   
1          1.0      0.0     0.0     0.0     0.0     1.0     0.0     0.0   
2          1.0      0.0     1.0     0.0     0.0     0.0     0.0     0.0   

   x4_Low  x4_Lvl  x5_AllPub  x5_NoSeWa  x6_Corner  x6_CulDSac  x6_FR2  \
0     0.0     1.0        1.0        0.0        0.0         0.0     0.0   
1     0.0     1.0        1.0        0.0        0.0         0.0     1.0   
2     0.0     1.0        1.0        0.0        0.0         0.0     0.0   

   x6_FR3  x6_Inside  x7_Gtl  x7_Mod  x7_Sev  x8_Blmngtn  x8_Blueste  \
0     0.0        1.0     1.0     0.0     0.0         0.0         0.0   
1     0.0        0.0     1.0     0.0     0.0         0.0         0.0   
2     0.0        1.0     1.0     0.0     0.0         0.0         0.0   

   x8_BrDale  x8_BrkSide  x8_ClearCr  x8_CollgCr  x8_Crawfor  x8_Edwards  \
0        0.0         0.0         0.0         1.0         0.0         0.0   
1        0.0         0.0         0.0         0.0         0.0         0.0   
2        0.0         0.0         0.0         1.0         0.0         0.0   

   x8_Gilbert  x8_IDOTRR  x8_MeadowV  x8_Mitchel  x8_NAmes  x8_NPkVill  \
0         0.0        0.0         0.0         0.0       0.0         0.0   
1         0.0        0.0         0.0         0.0       0.0         0.0   
2         0.0        0.0         0.0         0.0       0.0         0.0   

   x8_NWAmes  x8_NoRidge  x8_NridgHt  x8_OldTown  x8_SWISU  x8_Sawyer  \
0        0.0         0.0         0.0         0.0       0.0        0.0   
1        0.0         0.0         0.0         0.0       0.0        0.0   
2        0.0         0.0         0.0         0.0       0.0        0.0   

   x8_SawyerW  x8_Somerst  x8_StoneBr  x8_Timber  x8_Veenker  x9_Artery  \
0         0.0         0.0         0.0        0.0         0.0        0.0   
1         0.0         0.0         0.0        0.0         1.0        0.0   
2         0.0         0.0         0.0        0.0         0.0        0.0   

   x9_Feedr  x9_Norm  x9_PosA  x9_PosN  x9_RRAe  x9_RRAn  x9_RRNe  x9_RRNn  \
0       0.0      1.0      0.0      0.0      0.0      0.0      0.0      0.0   
1       1.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
2       0.0      1.0      0.0      0.0      0.0      0.0      0.0      0.0   

   x10_Artery  x10_Feedr  x10_Norm  x10_PosA  x10_PosN  x10_RRAe  x10_RRAn  \
0         0.0        0.0       1.0       0.0       0.0       0.0       0.0   
1         0.0        0.0       1.0       0.0       0.0       0.0       0.0   
2         0.0        0.0       1.0       0.0       0.0       0.0       0.0   

   x10_RRNn  x11_1Fam  x11_2fmCon  x11_Duplex  x11_Twnhs  x11_TwnhsE  \
0       0.0       1.0         0.0         0.0        0.0         0.0   
1       0.0       1.0         0.0         0.0        0.0         0.0   
2       0.0       1.0         0.0         0.0        0.0         0.0   

   x12_1.5Fin  x12_1.5Unf  x12_1Story  x12_2.5Fin  x12_2.5Unf  x12_2Story  \
0         0.0         0.0         0.0         0.0         0.0         1.0   
1         0.0         0.0         1.0         0.0         0.0         0.0   
2         0.0         0.0         0.0         0.0         0.0         1.0   

   x12_SFoyer  x12_SLvl  x13_Flat  x13_Gable  x13_Gambrel  x13_Hip  \
0         0.0       0.0       0.0        1.0          0.0      0.0   
1         0.0       0.0       0.0        1.0          0.0      0.0   
2         0.0       0.0       0.0        1.0          0.0      0.0   

   x13_Mansard  x13_Shed  x14_ClyTile  x14_CompShg  x14_Membran  x14_Metal  \
0          0.0       0.0          0.0          1

In [20]:
# checking for skewness 
skewed_features = [col for col in encoded_df.columns if encoded_df[col].skew() > 0.5]
print(len(skewed_features))

# taking log of skewed features increased the R2 from .93 to .94
encoded_df[skewed_features] = encoded_df[skewed_features].apply(lambda x: np.log1p(x))

252


In [24]:
# splitting into x and y and then scaling the x 

x = StandardScaler().fit_transform(encoded_df.drop('SalePrice',axis =1)) 
y = encoded_df['SalePrice']

# Training models 

In [26]:
# clf = RandomForestClassifier(n_estimators = 600,max_depth=2000, random_state=0)
# X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=42)
# clf.fit(X_train, y_train)

# # clf.predict(test_case.reshape(1, -1))
# print(clf.score(X_test, y_test))

In [27]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.linear_model import Ridge, LinearRegression, SGDRegressor, Lasso, ElasticNet
# from xgboost import XGBRegressor

lr = LinearRegression()
cv_score = cross_validate(lr, x, y,
                          cv = 10,
                          scoring = ['neg_root_mean_squared_error', 'neg_mean_squared_error'])
cv_score

{'fit_time': array([0.03386903, 0.01894951, 0.02092934, 0.01994562, 0.01994801,
        0.02892065, 0.02293897, 0.0279243 , 0.02198148, 0.02289796]),
 'score_time': array([0.0009973 , 0.00101161, 0.00099778, 0.00099826, 0.00100112,
        0.00099683, 0.00099707, 0.00099754, 0.00099683, 0.00099778]),
 'test_neg_root_mean_squared_error': array([-7.71437614e+11, -2.00852991e+12, -1.44350213e+12, -4.44425612e+11,
        -3.62185782e+12, -2.21509401e+11, -3.57426950e+12, -7.80876109e+11,
        -2.82667601e+12, -4.32320331e+11]),
 'test_neg_mean_squared_error': array([-5.95115992e+23, -4.03419238e+24, -2.08369840e+24, -1.97514125e+23,
        -1.31178541e+25, -4.90664147e+22, -1.27754025e+25, -6.09767498e+23,
        -7.99009725e+24, -1.86900868e+23])}

In [28]:
 cv_score['test_neg_root_mean_squared_error'].mean()

-1612540443783.9724

In [29]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(x, y)
model.score(x, y)

0.9463152407250803